# Compare model output with ASTE and observations

Load selected mooring time series (see choose_ASTE_profiles), and subsample CANARI LE accordingly

In [1]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import utils
import pickle

In [2]:
%matplotlib inline

Open all mooring time series and get grid points for subsetting CANARI-LE

In [5]:
import glob
mooring_files=glob.glob('../data/*tseries.nc')
mooring_vars={}
for file in mooring_files:
    mooring=xr.open_dataset(file)
    if 'iPROF' in mooring.dims:
        mooring=mooring.swap_dims({'iPROF':'time'})
    mooring=mooring.sortby('time').squeeze()
    if 'point' in mooring.dims:
        for ip,point in enumerate(mooring.point):
            mooring_vars[f'{mooring["loc"].values}_{ip}']=mooring[['prof_T','prof_Testim','prof_S','prof_Sestim']].sel(point=point)
    else:
        mooring_vars[str(mooring['loc'].values)]=mooring[['prof_T','prof_Testim','prof_S','prof_Sestim']]

Load NEMO T grid

In [6]:
import coast
nemo_dom = "/gws/nopw/j04/canari/users/dlrhodso/mesh_mask.nc"

config_grid={}
config_dir="../../tutorials/config"
for grid in ['t','f','u','v']:
    config_grid[grid]=f'{config_dir}/example_nemo_grid_{grid}.json'

nemo_t = coast.Gridded( fn_domain=nemo_dom, config=config_grid['t'])

/gws/smf/j04/canari/dask-env/lib/python3.10/site-packages/pydap/lib.py:5: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
/gws/smf/j04/canari/dask-env/lib/python3.10/site-packages/pkg_resources/__init__.py:2868: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('pydap')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
/gws/smf/j04/canari/dask-env/lib/python3.10/site-packages/pkg_resources/__init__.py:2868: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('pydap.responses')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
/gws/smf/j04/canari/dask-env/

Find NEMO grid points for all moorings

In [7]:
for mg_co in mooring_vars:
    mg_co_ds=mooring_vars[mg_co].squeeze()
    mean_lat=mg_co_ds.prof_lat.mean('time')
    mean_lon=mg_co_ds.prof_lon.mean('time')
    [mg_co_ds['yy'],mg_co_ds['xx']]=nemo_t.find_j_i(lat=mean_lat,lon=mean_lon)
    mooring_vars[mg_co]=mg_co_ds
    print(mg_co_ds.time.min('time').data,mg_co_ds.time.max('time').data)
mooring_vars

2005-08-30T00:00:00.000000000 2016-10-11T00:00:00.000000000
2007-09-02T00:00:00.000000000 2011-07-14T00:00:00.000000000
2004-09-27T00:00:00.000000000 2013-09-17T00:00:00.000000000
2002-08-13T00:00:00.000000000 2012-06-27T00:00:00.000000000
2014-08-12T00:00:00.000000000 2016-08-08T00:00:00.000000000
2014-08-12T00:00:00.000000000 2016-08-08T00:00:00.000000000


{'Beaufort Mooring': <xarray.Dataset>
 Dimensions:              (time: 3007)
 Coordinates: (12/21)
     prof_descr           (time) |S30 ...
     prof_date            (time) float64 ...
     prof_YYYYMMDD        (time) float64 ...
     prof_HHMMSS          (time) float64 ...
     prof_lon             (time) float64 -140.0 -140.0 -140.0 ... -140.1 -140.1
     prof_lat             (time) float64 74.0 74.0 74.0 74.0 ... 74.0 74.0 74.0
     ...                   ...
     prof_interp_lat      (time) float64 ...
     prof_interp_weights  (time) float64 ...
   * time                 (time) datetime64[ns] 2005-08-30 ... 2016-10-11
     depth                float64 ...
     loc                  <U16 'Beaufort Mooring'
     point                float64 1.134e+06
 Data variables:
     prof_T               (time) float64 ...
     prof_Testim          (time) float64 ...
     prof_S               (time) float64 ...
     prof_Sestim          (time) float64 ...
     yy                   int64 1114
   

Load all LE members

In [8]:
%%time
import dask_gateway
import dask
# Create a connection to dask-gateway.
gw = dask_gateway.Gateway("https://dask-gateway.jasmin.ac.uk", auth="jupyterhub")

# Inspect and change the options if required before creating your cluster.
options = gw.cluster_options()
options.worker_cores = 1 #keeping this at 1 and allowing 15 worker processes seems to run faster than the other way around
options.scheduler_cores = 1 #we need at least one core for the scheduler
#specify which conda env to use, this must match the versions of python and dask (and a few other libraries) used on the notebook service
options.worker_setup='source /apps/jasmin/jaspy/mambaforge_envs/jaspy3.10/mf-22.11.1-4/bin/activate /gws/smf/j04/canari/dask-env'

# Create a dask cluster, or, if one already exists, connect to it.
# This stage creates the scheduler job in SLURM, so may take some time.
# While your job queues.
clusters = gw.list_clusters()
if not clusters:
    cluster = gw.new_cluster(options, shutdown_on_close=False)
else:
    cluster = gw.connect(clusters[0].name)

# Create at least one worker, and allow your cluster to scale to 15.
# The max JASMIN allows is 16, but one of these is used as the scheduler.
cluster.adapt(minimum=1, maximum=15)

# Get a dask client.
client = cluster.get_client()

CPU times: user 177 ms, sys: 43 ms, total: 220 ms
Wall time: 12.9 s


In [9]:
data_path = "/gws/nopw/j04/canari/shared/large-ensemble/priority/HIST2"
ensembles=glob.glob(f'{data_path}/*')
for ens_file in ensembles:
    ens=int(ens_file.split('/')[-1])
    if glob.glob(f'../data/mooring_tseries_ens{ens}.nc'):
        print('File exists, skipping')
        continue
    else:
        print(f'Doing ensemble member {ens}')
    ens_data_path=f'{data_path}/{ens}/OCN/yearly'
    Tdata=[]
    for year in range(2000,2015):
        temp_Tgrid = glob.glob(f"{ens_data_path}/{year}/*_mon__grid_T_votemper.nc")
        sal_Tgrid = glob.glob(f"{ens_data_path}/{year}/*_mon__grid_T_vosaline.nc")
        try:
            Tdata.append(xr.open_mfdataset([temp_Tgrid[0],sal_Tgrid[0]]))
        except:
            print(f'No data for member {ens}, year {year}')
            continue
    if Tdata:
        Tdata=xr.concat(Tdata,'time_counter')
    else:
        print(f'Tdata empty, skipping {ens} entirely')
        continue
    datetimeindex = Tdata.indexes['time_counter'].to_datetimeindex()
    Tdata['time_counter']=datetimeindex
    Tdata_allm=[]
    for loc in mooring_vars:
        mdata=mooring_vars[loc]
        Tdata_mooring=Tdata.sel(x=mdata.xx,y=mdata.yy)
        Tdata_mooring=Tdata_mooring.interp(deptht=mdata.depth)
        Tdata_allm.append(Tdata_mooring)
    Tdata_allm=xr.concat(Tdata_allm,'loc')
    print('Writing mooring t series to file')
    Tdata_allm.drop_vars('time_counter_bounds').to_netcdf(f'../data/mooring_tseries_ens{ens}.nc')

File exists, skipping
File exists, skipping
File exists, skipping
File exists, skipping
File exists, skipping
File exists, skipping
File exists, skipping
File exists, skipping
Doing ensemble member 21


IndexError: list index out of range